## Memory and Compute Bound

![](./roofline.png)

A particular kernel's compute intensity (FLOPs per Byte move) and hardware determines the upperbound performance (FLOPs per second).

When compute intensity is smaller (before the "ridge point"), it is mainly bounded by memory speed (Byte per second):
$$
\text{FLOP} / \text{Second} = \frac{\text{FLOP}}{\text{Byte}} \times \frac{\text{Byte}}{\text{Second}} 
$$

By improving algorithm and adding compute intensity (e.g. vectorization), we move over the ridge point and since that the main bound becomes compute speed: 
$$
\text{FLOP} / \text{Second} = \min(\frac{\text{FLOP}}{\text{Byte}} \times \frac{\text{Byte}}{\text{Second}} , \text{Peak} \frac{\text{FLOP}}{\text{Byte}}) 
$$

## Program Parallism vs. Resource Occupancy

Assume an instruction without load/store can execute 1 op per 2 cycles, and an instruction with load/store will go down to 1 op per 400 cycles.
If the load or store instruction are independent and can be parallelized (such as vector element additions), obviously, it is not smart to queue these instructions and execute them in order.

Instead, we can exploit the **program parallism** and *hide the latency* by making 200 parallel execution units and parallel threads, so that we take 400 cycles to execute 200 concurrent instructions resulting in an 1 op per 2 cycle *throughputs*.
Assume you wait a minimum of 2 cycles before checking results, you will find no difference between the load/store instruction and a normal instruction, thus we **"hide"** the "latency" from you.

Of course, in reality, GPU has a limited number of SM cores and compute units, and algorithm is not always fully parallelizable. As a result, we may want to greedy issue as many Warps as possible?

The answer is NO! In a blog post by Modular [1], they try to obtain the SoTA NF4 Dequantization kernel results on a pretty old GPU (Tesla T4), and the single most important step they manage to get there is to **reduce** the block size from 1024 threads to 512 threads.

As we know,
* if we set block size = 1024, we will end up 1024 / 32 = 32 Warps in the scheduled SM (a block cannot be breaked into multiple SMs);
* if we set block size = 512, we will end up 512 / 32 = 16 Warps instead.

And using less Warps apparently violates our intuition to create more parallel Warps (GPU can do "scoreboard" scheduling [2], and execute parallelizable Warps when compute units are available and its data dependency is met). Why? Because your goal is actually to have more Warps total on the SM. Reducing the Warps per Block is just the counter-intuitive tactic to achieve that goal.

The real culprit is **resource occupancy**: because Tesla T4 is an old GPU, it has a high register pressure with the given thread workload.

Because registers are strictly private to each individual thread in a GPU, more threads a Warp has, more register space the Warp will occupy.
As a result, a 32-Warp block will consumes more register file space than a 16-Warp block.
And if the 32-Warp block uses too much resources (like in the Tesla T4 case), no more extra Warps will be allowed to join the execution on this particular SM (when this block of warps stall on memory, nothing else can fill the bubble to hide the latency).

Assume Tesla T4 allows maximum 64 Warps to be concurrently scheduled on an SM, our high-demanding 32-Warp -- in the extreme case -- prevent any extra Warps to join the execution, resulting in a 32/64 = 50% occupancy.

In a heavy work load where all SM will be utilized anyway, a 50% occupancy is far worse than a higher occupancy achieved by lowering the block size to 16 Warps. 

*Footnote*: 
1. Compared to the Tomasulo algorithm, scoreboard scheduling is more simple to implement in hardware, as it does not involve result-forwarding bus (instead, it simply writes results back to the register), register renaming (historically not used, but can be added), and branch parallism.
2. What about the other extreme -- we set the block size to only 32 so that many small blocks can be stacked on a SM to occupy the resources? The answer is -- No, in additional to the synchronization convenience (e.g., barrier, shared register and memory) a block can offer to its threads, (the block size in number of warps) $\times$ (the maximum blocks per SM) should be $\ge$ (the maximum warps per SM) to achieve 100% occupancy. Having a block size of 32 is not likely going to achieve this by hardware design.

## Reference
[1] https://www.modular.com/blog/how-to-beat-unsloth-s-cuda-kernel-using-mojo-with-zero-gpu-experience

[2] https://patents.google.com/patent/US7676657 Nvidia Pattern: *Across-thread out-of-order instruction dispatch in a multithreaded microprocessor*.